In [1]:
import pywt
import os
from pylab import *
from numpy import*
import pandas as pd

#### Обработка графиков ЭКГ и преобразование в таблицу

In [2]:
def get_ecg(key):
  way = 'train/' + key + '.npy'
  ecgsig = np.load(way).transpose().reshape((60000,))
  x = np.linspace(0,  1,  num = 2048)
  data = np.sin(250 * np.pi * x**2)
  coefs = pywt.downcoef('a', data, 'db20', mode='symmetric', level=1)
  st='sym5'
  coeffs = pywt.wavedec(ecgsig, st, level=5)
  row = coeffs[0]
  return row

In [3]:
def into_one_table(keys):
  row = get_ecg(keys[0])
  row = np.insert(row, 0, int(keys[0].strip('_hr')))
  table = [row]
  for i in range(target.index.start + 1, target.index.stop):
    row = get_ecg(keys[i])
    row = np.insert(row, 0, int(keys[i].strip('_hr')))
    table = np.vstack([table, row])
  table = pd.DataFrame(table)
  return table

In [4]:
target = pd.Series(list(map(lambda x: x[:-4], os.listdir('train')[:-2])))
ECG_data = into_one_table(target)

In [5]:
ECG_data[0] = ECG_data[0].astype('int')
ECG_data = ECG_data.rename(columns={0: 'ecg_id'})

In [6]:
ECG_data = ECG_data.sort_values(by=['ecg_id'], ascending=True)

In [7]:
ECG_data

,ecg_id,1,2,3,4,5,6,7,8,9,...,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883
0,9,-0.170442,-0.066933,-0.140079,-0.478502,-1.245167,-0.979738,-0.738943,-0.476043,-0.349288,...,0.558936,0.551860,0.551007,0.561618,0.556675,0.645774,0.744957,0.649421,0.667106,0.731759
1,34,0.399648,0.579593,0.460080,-0.077166,-1.270282,-1.040847,-1.689153,-2.481032,-2.205554,...,1.113606,1.088111,1.099680,1.132341,1.130250,1.788834,2.455783,1.818630,1.938019,2.363243
2,43,0.411589,0.365335,0.399006,0.554728,0.911140,0.774487,0.576258,0.404740,0.229235,...,0.341701,0.340029,0.342162,0.341734,0.340499,0.346990,0.366654,0.343959,0.348800,0.365320
3,52,-0.162251,-0.159025,-0.159327,-0.163111,-0.158814,-0.159398,-0.162262,-0.174764,-0.179583,...,-0.267047,-0.258175,-0.261318,-0.274144,-0.273755,-0.519687,-0.765086,-0.532849,-0.575938,-0.730437
4,57,-0.268330,-0.279812,-0.270770,-0.232069,-0.136626,-0.155710,-0.117283,-0.096148,-0.074360,...,-0.304742,-0.297821,-0.302716,-0.310384,-0.311176,-0.513279,-0.714933,-0.521431,-0.558040,-0.686983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,21784,-0.094946,-0.097736,-0.095932,-0.087848,-0.070772,-0.076037,-0.082161,-0.089443,-0.092388,...,0.005842,0.004748,0.005003,0.006885,0.006829,0.041791,0.079657,0.044349,0.050701,0.074361
2097,21795,-0.194416,-0.182553,-0.191956,-0.238293,-0.352612,-0.314071,-0.319038,-0.313175,-0.285890,...,0.120157,0.117738,0.121368,0.121892,0.122540,0.189928,0.260724,0.190226,0.204078,0.251573
2098,21825,-0.004072,-0.000198,-0.003268,-0.017719,-0.054790,-0.048271,-0.088630,-0.084493,-0.070101,...,-0.128999,-0.124998,-0.127924,-0.131497,-0.131004,-0.222249,-0.318699,-0.224482,-0.242503,-0.305902
2099,21831,-0.128454,-0.127641,-0.128944,-0.136319,-0.158699,-0.147968,-0.147763,-0.142817,-0.138197,...,-0.159457,-0.155589,-0.158779,-0.161389,-0.160510,-0.233738,-0.313892,-0.233884,-0.249503,-0.303710


## N-граммы

In [442]:
import neurokit2
import sleepecg
import wfdb.processing
from nltk.util import ngrams

from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [9]:
def find_peaks(ecgsignal, peak):
    return wfdb.processing.correct_peaks(ecgsignal, rpeaks,
            search_radius=36, smooth_window_size=50, peak_dir=peak)

In [10]:
def get_cycle(ecgsignal, rpeaks): # разбиение ЭКГ на циклы
  ecgsignal_split = []
  cycle = []
  rpeaks = rpeaks[::-1]
  f = True #флаг конца массива с пиками
  f1 = True #флаг вхождения первого пика
  for i in range(len(ecgsignal)):
    if f and i == rpeaks[-1]:
      if f1:
        f1 = False
        cycle = []
      else:
        ecgsignal_split.append(cycle)
        cycle = []
      rpeaks = rpeaks[:-1]
      if f and len(rpeaks) == 0:
        f = False
    cycle.append(ecgsignal[i])
  return ecgsignal_split

In [11]:
def sign(x):
   return 1 if x > 0 else -1

In [12]:
def RTL(ecgsignal_split):
    sp = [[(i[0] - min(i[:10])), len(i), ((i[0] - min(i[:10]))/len(i))] for i in ecgsignal_split]
    s = []
    for i in range(len(sp) - 1):
        R = sp[i + 1][0] - sp[i][0]
        T = sp[i + 1][1] - sp[i][1]
        L = sp[i + 1][2] - sp[i][2]
        s.append([R, T, L])
    return s

In [13]:
def letters(data): # кодирование цикла по В.М. Успенскому 
  if data[0] == 1:
    if data[1] == -1:
      return 'C'
    else:
      if data[2] == -1:
        return 'E'
      return 'A'
  else:
    if data[1] == 1:
      return 'D'
    else:
      if data[2] == 1:
        return 'B'
      return 'F'

In [14]:
def encoding(ecgsignal_split):
    s = [[sign(i) for i in k] for k in RTL(ecgsignal_split)]
    stroka = ''
    for i in s:
        stroka += letters(i)
    return stroka

In [16]:
ECG_data_array = np.array(ECG_data)

In [17]:
ecg_ids = []
encoding_ecgs = []

In [18]:
for ecgsignal in ECG_data_array:
    ecg_id = ecgsignal[0]
    ecgsignal = ecgsignal[1:]
    
    _, results = neurokit2.ecg_peaks(ecgsignal, sampling_rate=100)
    rpeaks = results["ECG_R_Peaks"]
    rpeaks = sleepecg.detect_heartbeats(ecgsignal, fs=100)
    rpeaks = wfdb.processing.xqrs_detect(ecgsignal, fs=100, verbose=False)
    
    rpeaks = find_peaks(ecgsignal, 'up')
    
    ecgsignal_split = get_cycle(ecgsignal, rpeaks)
    encoding_ecg = encoding(ecgsignal_split)
    
    ecg_ids.append(ecg_id)
    encoding_ecgs.append(encoding_ecg)

In [19]:
encoding_ecg = pd.Series(encoding_ecgs, index=ecg_ids)

In [21]:
gts = pd.read_csv('train/train_gts.csv')
gts.record_name = gts.record_name.map(lambda x: float(x[:-3]))
gts = gts.sort_values(by='record_name')
gts.insert(2, 'encoding_ecg', array(encoding_ecg))
gts

,record_name,myocard,encoding_ecg
265,9.0,0,DDCADDCC
1407,34.0,0,DDDCCCFFFDDFCCCACFE
1833,43.0,0,BFABCFADDEC
441,52.0,0,DDEFBCCBCCF
1834,57.0,0,DDCADBCDD
...,...,...,...
1753,21784.0,0,DDCCFFAFAFFAFC
2008,21795.0,0,
72,21825.0,0,ECCDDDECB
2100,21831.0,0,DAFCCDDDCFCD


In [22]:
sick = gts[gts.myocard == 1].encoding_ecg
healthy = gts[gts.myocard == 0].encoding_ecg

In [158]:
sp_sick= []
for i in sick:
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(i), 6))))
    for k in n:
        sp_sick.append(k)

In [159]:
sp_healthy= []
for i in healthy:
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(i), 6))))
    for k in n:
        sp_healthy.append(k) 

In [160]:
def counter_and_sort(arr):
  quantity = {}
  for i in arr:
    if i in quantity:
      quantity[i] += 1
    else:
      quantity[i] = 1
  quantity = pd.Series(list(quantity.values()), index=list(quantity.keys())).sort_values(ascending=False)
  return quantity

In [161]:
q_sick = counter_and_sort(sp_sick)
q_health = counter_and_sort(sp_healthy)

In [162]:
only_sick = list(set(q_sick.index) - set(q_health.index))
only_health = list(set(q_health.index) - set(q_sick.index))

In [520]:
popular_sick = q_sick.iloc[:len(q_sick) // 2]
popular_health = q_health.iloc[:len(q_health) // 2]

In [521]:
popular_sick_only = list(set(popular_sick.index) - set(popular_health.index))
popular_health_only = list(set(popular_health.index) - set(popular_sick.index))

In [522]:
ngramms_ecgs = []
for ecg in array(gts):
    ngramms_ecg = set()
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(ecg[2]), 6))))
    for k in n:
        ngramms_ecg.add(k)
    ngramms_ecgs.append(list(ngramms_ecg))

In [523]:
def func(ng):
    ng_health = sum(list(map(lambda x: 1 if x in only_health else 0, ng)))
    ng_sick = sum(list(map(lambda x: 1 if x in only_sick else 0, ng)))
    
    ng_popular_health = sum(list(map(lambda x: 1 if x in popular_health_only else 0, ng)))
    ng_popular_sick = sum(list(map(lambda x: 1 if x in popular_sick_only else 0, ng)))
    
    return [ng_health, ng_sick, ng_popular_health, ng_popular_sick]

In [524]:
arr = pd.DataFrame([func(ng) for ng in ngramms_ecgs])

In [525]:
X_train, X_test, y_train, y_test = train_test_split(arr, gts.myocard, random_state=0, test_size=0.75)

In [526]:
cat = CatBoostClassifier(depth=9,  learning_rate=0.03)
cat.fit(X_train, y_train)

0:	learn: 0.6444023	total: 7.48ms	remaining: 7.47s
1:	learn: 0.6138423	total: 12.4ms	remaining: 6.17s
2:	learn: 0.5774073	total: 18ms	remaining: 5.97s
3:	learn: 0.5306942	total: 21.3ms	remaining: 5.31s
4:	learn: 0.4964677	total: 27.5ms	remaining: 5.48s
5:	learn: 0.4700498	total: 33.9ms	remaining: 5.61s
6:	learn: 0.4540456	total: 39.2ms	remaining: 5.56s
7:	learn: 0.4295515	total: 44.9ms	remaining: 5.57s
8:	learn: 0.4147525	total: 50.5ms	remaining: 5.56s
9:	learn: 0.3949160	total: 57.7ms	remaining: 5.71s
10:	learn: 0.3697493	total: 62ms	remaining: 5.57s
11:	learn: 0.3527891	total: 69.5ms	remaining: 5.72s
12:	learn: 0.3357151	total: 76.7ms	remaining: 5.82s
13:	learn: 0.3178629	total: 83.7ms	remaining: 5.9s
14:	learn: 0.3044144	total: 90.9ms	remaining: 5.97s
15:	learn: 0.2972428	total: 97.8ms	remaining: 6.01s
16:	learn: 0.2931872	total: 103ms	remaining: 5.98s
17:	learn: 0.2835690	total: 110ms	remaining: 6s
18:	learn: 0.2761392	total: 116ms	remaining: 5.99s
19:	learn: 0.2699614	total: 122ms

194:	learn: 0.1652736	total: 832ms	remaining: 3.43s
195:	learn: 0.1652667	total: 836ms	remaining: 3.43s
196:	learn: 0.1652427	total: 841ms	remaining: 3.43s
197:	learn: 0.1651861	total: 845ms	remaining: 3.42s
198:	learn: 0.1651782	total: 849ms	remaining: 3.42s
199:	learn: 0.1651742	total: 853ms	remaining: 3.41s
200:	learn: 0.1651257	total: 858ms	remaining: 3.41s
201:	learn: 0.1651141	total: 862ms	remaining: 3.41s
202:	learn: 0.1650525	total: 867ms	remaining: 3.4s
203:	learn: 0.1650485	total: 872ms	remaining: 3.4s
204:	learn: 0.1650316	total: 875ms	remaining: 3.39s
205:	learn: 0.1649592	total: 880ms	remaining: 3.39s
206:	learn: 0.1649432	total: 883ms	remaining: 3.38s
207:	learn: 0.1649348	total: 887ms	remaining: 3.38s
208:	learn: 0.1649182	total: 890ms	remaining: 3.37s
209:	learn: 0.1648925	total: 895ms	remaining: 3.37s
210:	learn: 0.1648878	total: 897ms	remaining: 3.35s
211:	learn: 0.1648712	total: 902ms	remaining: 3.35s
212:	learn: 0.1647960	total: 905ms	remaining: 3.34s
213:	learn: 0.

380:	learn: 0.1608899	total: 1.65s	remaining: 2.68s
381:	learn: 0.1608808	total: 1.65s	remaining: 2.68s
382:	learn: 0.1608721	total: 1.66s	remaining: 2.67s
383:	learn: 0.1608653	total: 1.66s	remaining: 2.67s
384:	learn: 0.1608568	total: 1.67s	remaining: 2.67s
385:	learn: 0.1608483	total: 1.67s	remaining: 2.66s
386:	learn: 0.1608409	total: 1.68s	remaining: 2.66s
387:	learn: 0.1608333	total: 1.68s	remaining: 2.65s
388:	learn: 0.1608250	total: 1.69s	remaining: 2.65s
389:	learn: 0.1608179	total: 1.69s	remaining: 2.65s
390:	learn: 0.1608099	total: 1.7s	remaining: 2.64s
391:	learn: 0.1608024	total: 1.7s	remaining: 2.64s
392:	learn: 0.1607943	total: 1.71s	remaining: 2.64s
393:	learn: 0.1607868	total: 1.71s	remaining: 2.63s
394:	learn: 0.1607810	total: 1.72s	remaining: 2.63s
395:	learn: 0.1607731	total: 1.72s	remaining: 2.63s
396:	learn: 0.1607660	total: 1.73s	remaining: 2.62s
397:	learn: 0.1607590	total: 1.73s	remaining: 2.62s
398:	learn: 0.1607534	total: 1.73s	remaining: 2.61s
399:	learn: 0.

554:	learn: 0.1601410	total: 2.46s	remaining: 1.97s
555:	learn: 0.1601388	total: 2.46s	remaining: 1.97s
556:	learn: 0.1601367	total: 2.47s	remaining: 1.96s
557:	learn: 0.1601344	total: 2.47s	remaining: 1.96s
558:	learn: 0.1601323	total: 2.48s	remaining: 1.95s
559:	learn: 0.1601302	total: 2.48s	remaining: 1.95s
560:	learn: 0.1601279	total: 2.48s	remaining: 1.94s
561:	learn: 0.1601255	total: 2.49s	remaining: 1.94s
562:	learn: 0.1601231	total: 2.49s	remaining: 1.94s
563:	learn: 0.1601211	total: 2.5s	remaining: 1.93s
564:	learn: 0.1601189	total: 2.5s	remaining: 1.93s
565:	learn: 0.1601169	total: 2.51s	remaining: 1.92s
566:	learn: 0.1601146	total: 2.51s	remaining: 1.92s
567:	learn: 0.1601126	total: 2.52s	remaining: 1.92s
568:	learn: 0.1601105	total: 2.52s	remaining: 1.91s
569:	learn: 0.1601082	total: 2.53s	remaining: 1.91s
570:	learn: 0.1601062	total: 2.53s	remaining: 1.9s
571:	learn: 0.1601043	total: 2.54s	remaining: 1.9s
572:	learn: 0.1601023	total: 2.54s	remaining: 1.89s
573:	learn: 0.16

728:	learn: 0.1598707	total: 3.29s	remaining: 1.22s
729:	learn: 0.1598695	total: 3.29s	remaining: 1.22s
730:	learn: 0.1598684	total: 3.29s	remaining: 1.21s
731:	learn: 0.1598674	total: 3.3s	remaining: 1.21s
732:	learn: 0.1598661	total: 3.3s	remaining: 1.2s
733:	learn: 0.1598651	total: 3.31s	remaining: 1.2s
734:	learn: 0.1598639	total: 3.31s	remaining: 1.19s
735:	learn: 0.1598629	total: 3.32s	remaining: 1.19s
736:	learn: 0.1598619	total: 3.32s	remaining: 1.19s
737:	learn: 0.1598608	total: 3.33s	remaining: 1.18s
738:	learn: 0.1598599	total: 3.33s	remaining: 1.18s
739:	learn: 0.1598589	total: 3.33s	remaining: 1.17s
740:	learn: 0.1598578	total: 3.34s	remaining: 1.17s
741:	learn: 0.1598569	total: 3.34s	remaining: 1.16s
742:	learn: 0.1598560	total: 3.35s	remaining: 1.16s
743:	learn: 0.1598548	total: 3.35s	remaining: 1.15s
744:	learn: 0.1598536	total: 3.36s	remaining: 1.15s
745:	learn: 0.1598524	total: 3.36s	remaining: 1.15s
746:	learn: 0.1598515	total: 3.37s	remaining: 1.14s
747:	learn: 0.15

901:	learn: 0.1597246	total: 4.11s	remaining: 446ms
902:	learn: 0.1597241	total: 4.11s	remaining: 441ms
903:	learn: 0.1597234	total: 4.11s	remaining: 437ms
904:	learn: 0.1597225	total: 4.12s	remaining: 432ms
905:	learn: 0.1597219	total: 4.12s	remaining: 428ms
906:	learn: 0.1597212	total: 4.13s	remaining: 423ms
907:	learn: 0.1597206	total: 4.13s	remaining: 419ms
908:	learn: 0.1597199	total: 4.14s	remaining: 414ms
909:	learn: 0.1597193	total: 4.14s	remaining: 410ms
910:	learn: 0.1597187	total: 4.15s	remaining: 405ms
911:	learn: 0.1597181	total: 4.15s	remaining: 401ms
912:	learn: 0.1597175	total: 4.16s	remaining: 396ms
913:	learn: 0.1597169	total: 4.16s	remaining: 392ms
914:	learn: 0.1597162	total: 4.16s	remaining: 387ms
915:	learn: 0.1597155	total: 4.17s	remaining: 382ms
916:	learn: 0.1597150	total: 4.17s	remaining: 378ms
917:	learn: 0.1597144	total: 4.18s	remaining: 373ms
918:	learn: 0.1597138	total: 4.18s	remaining: 369ms
919:	learn: 0.1597132	total: 4.19s	remaining: 364ms
920:	learn: 

In [527]:
f1_score(y_test, cat.predict(X_test))

0.7657841140529531

In [528]:
f1_score(y_train, cat.predict(X_train))

0.7941176470588235